# LLM week 3 Homework 

-- by Junjie Zhang

Assignment: Implement a RAG-based Chatbot using Langchain


-   • A console program to continuously receive user input and respond
-   • Try smaller dataset first and then larger dataset
-   • Refer to the examples in


## Summary

I loaded 11 literatures from the local disk using a directory loader for RAG and chatbot. THe contents are about genetic breeding which is used for seeds innovation. 

**This homework includes the following steps:**
1. **Environment Setup**
    - Import necessary libraries and modules.
    - Load environment variables from `.env` file to get the API key.

2. **Document Processing**
    - Extract text from multiple PDF files and save each to a separate text file.
    - Load documents from the local disk using a directory loader.
    - Split documents into smaller chunks for better processing.

3. **Embedding Creation**
    - Create embeddings for each chunk using OpenAI embeddings.
    - Create a retriever from the vector store to fetch relevant chunks.

4. **Conversational Chain Setup**
    - Initialize the chat model.
    - Create a memory object to store the conversation history.
    - Create a conversational retrieval chain by combining the chat model, retriever, and memory.

5. **User Interaction**
    - Start an infinite loop to interact with the user.
    - Continuously receive user input and respond using the conversational retrieval chain.
    - Exit the loop if the user inputs "exit".

**Challenges:**
1. **Document Processing:** I had difficulty in reading the PDF files by DirectoryLoader. 
2. **performance:** The performance of the chatbot is not good enough. It can retrieved the relevant information but it is not deep enough. May need more data and better model to improve the performance.



In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')

In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
import os
os.getcwd()

'/Users/junjiezhang/Documents/GitHub/LLM_Zeng24/homework'

In [ ]:
# import glob
# import PyPDF2

# # Extract the text from multiple PDF files and write each to a separate text file
# def extract_text_from_pdfs():
#     context_files = glob.glob("../files/genetics_breeding/*.pdf")
#     for file_path in context_files:
#         output_file_path = file_path.replace(".pdf", ".txt")
#         with open(file_path, 'rb') as file:
#             reader = PyPDF2.PdfReader(file)
#             text = ""
#             for page_num in range(len(reader.pages)):
#                 page = reader.pages[page_num]
#                 text += page.extract_text() + "\n"
#             with open(output_file_path, 'w') as txt_file:
#                 txt_file.write(text)

# extract_text_from_pdfs()


In [ ]:
#pip install --upgrade pypdf langchain

In [15]:

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
# Load all PDF documents from the local directory
loader = PyPDFDirectoryLoader('../files/genetics_breeding', glob="*.pdf")
documents = loader.load()


Ignoring wrong pointing object 2588 0 (offset 0)
Ignoring wrong pointing object 2592 0 (offset 0)
Ignoring wrong pointing object 4257 0 (offset 0)
Ignoring wrong pointing object 4258 0 (offset 0)
Ignoring wrong pointing object 4259 0 (offset 0)
Ignoring wrong pointing object 4260 0 (offset 0)
Ignoring wrong pointing object 4261 0 (offset 0)
Ignoring wrong pointing object 4262 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)


In [19]:
documents[0]

Document(metadata={'source': '../files/genetics_breeding/Agronomy Journal - 2015 - Casler - Fundamentals of Experimental Design  Guidelines for Designing Successful Experiments.pdf', 'page': 0}, page_content='692\t Agronomy\tJournal\t •\t Volume\t107,\tIssue\t2\t •\t 2015\nSymposium: Statistical Concepts\nFundamentals\tof\tExperimental\tDesign:\tGuidelines\tfor\t\nDesigning\tSuccessful\tExperiments\nMichael\tD.\tCasler*\nPublished in Agron. J. 107:692–705 (2015)\ndoi:10.2134/agronj2013.0114\nAvailable freely online through the author-supported open access option.\nCopyright © 2015 by the American Society of Agronomy, 5585 Guilford \nRoad, Madison, WI 53711. All rights reserved. No part of this periodical may \nbe reproduced or transmitted in any form or by any means, electronic or \nmechanical, including photocopying, recording, or any information storage \nand retrieval system, without permission in writing from the publisher.\nDesigning\texperiments\tinvolves\ta\tmarriage of biologi-

In [ ]:
# some pdf files have problem to load, use this code to load one by one to identify the problem files. 
# import os
# from langchain.document_loaders import PyPDFLoader

# directory = '../files/genetics_breeding'

# # List all PDF files in the directory
# pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

# # Check each PDF file
# for pdf_file in pdf_files:
#     try:
#         pdf_path = os.path.join(directory, pdf_file)
#         loader = PyPDFLoader(pdf_path)
#         documents = loader.load()
#         print(f"{pdf_file}: Loaded successfully with {len(documents)} documents.")
#     except Exception as e:
#         print(f"{pdf_file}: Failed to load. Error: {e}")

In [4]:

# print("Loading documents")
# # Load documents from the local disk using a directory loader
# loader = DirectoryLoader('../files/genetics_breeding', glob="*.txt")
# documents = loader.load()


In [20]:
# Split documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print("Creating embeddings")

# Create embeddings for each chunk using OpenAI embeddings
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, embeddings)

# Create a retriever from the vector store to fetch relevant chunks
retriever = vectorstore.as_retriever()

print("Creating chains")

Creating embeddings
Creating chains


In [ ]:
import textwrap

# Initialize the chat model
llm = ChatOpenAI()

# Create a memory object to store the conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create a conversational retrieval chain by combining the chat model, retriever, and memory
conversation = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, memory=memory, verbose=True
)

while True:
    user_input = input("> ")  # Get user input
    if user_input.lower() == "exit":  # Check if the user wants to exit
        print("Exiting the conversation.")
        break  # Exit the loop
    result = conversation.invoke(user_input)  # Invoke the conversation chain with the user input
    wrapped_result = textwrap.fill(result["answer"], width=80)
    print(wrapped_result)
    #print("\n".join(result["answer"][i:i+80] for i in range(0, len(result["answer"]), 80)))  # Print the chatbot's response with word wrapping



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
ii

●
●
●
●
●
●●● ●●
●
● ●● ● ●
●
●
●
●
●●
● ●● ●
●
● ●●
●●
●●
●
●
●
●
●
●
●
●●
●
●
●
●
●
●
●● ●● ● ●
●● ● ●
●●●
●
●
●● ●
●● ●
● ●
●●
●● ●● ● ●
●
●●●
● ●
●
●●
●
●
●
●
●
●
●
●
●
●●
● ●
●
●●
●● ●●●
●●
●●
●
●
●
●
●●
●
● ●
●
●
●
●
●
●
●
● ●
●
●
● ●
● ●
●
●
●
●
●
●●
●
●
●●
●
●
●
●
● ●
● ●●
●
●● ●
●
●
● ●
●
●
●
●● ● ●
●●
●
●● ●
● ●●
●
●
●
●
● ●
●● ●●●
●
● ●
●
●●
●
●● ●
●●
●
●
● ●
●●
●●●
●
●
● ●●
● ●●
●
●
●
● ●
●
●
●
●
●
●●
●
●
●● ●●● ●
●●
● ●
●
●
●
●
●
●
●●
●
● ●
●●●
●
● ●
●●
●●
● ● ●
●
●
●
●
●
●
● ● ●
●●
●
●●
●
●
●
●
● ●
●● ●
●●
●
●
●
●●
●
●
● ●●
●
● ●
●● ●
●●
●
●
●
●
●
●
●
●
●●● ●
●
●
●●
● ●
●
●●
● ●● ●
●
●●
●
●
●
●● ● ●
●
●
●
●
●
● ●
●
●
●
●
●
● ●
●
● ●● ●
●
●
●
●
●
● ●
●
●
● ●●●
●
●
●
●
●
●
●
●● ● ●
●●